In [1]:
from decoder.geoutils import *
from decoder import Decoder
from rich import print
import numpy as np

In [2]:
decoder = Decoder()
data = decoder.load(
    "Test_Data/datos_asterix_radar.ast",
    parallel=False,
    max_messages=10,
)
print(data[1])

Loaded 133 bytes from Test_Data/datos_asterix_radar.ast


Decoding:   0%|          | 0/133 [00:00<?, ?it/s]
Decoding: 0Msg [00:00, ?Msg/s]


IndexError: list index out of range

In [18]:
rho = data[1]["Range m"]
theta = data[1]["Theta"] * np.pi / 180
H = data[1]["Height (m)"]

radar_lat = (41 + 18 / 60.0 + 2 / 3600.0)*np.pi/180
radar_lon = (2 + 6 / 60.0 + 7 / 3600.0)*np.pi/180
radar_alt = 27
coords_radar = CoordinatesWGS84(radar_lat, radar_lon, radar_alt)

Rt=6378e3

In [19]:
elevation = np.asin((H - radar_alt) / rho)
coords_polar = CoordinatesPolar(rho, theta, elevation)

In [20]:
coords_cartesian = GeoUtils.change_radar_spherical_2_radar_cartesian(coords_polar)

In [21]:
coords_geocentric = GeoUtils().change_radar_cartesian_2_geocentric(
    coords_radar, coords_cartesian
)

In [22]:
coords_geodesic = GeoUtils().change_geocentric_2_geodesic(coords_geocentric)
print(str(coords_geodesic))

41:16:40.0837N 002:04:48.2477E 96.5015m
lat:41.277801024 lon:2.080068817

In [24]:
b=Rt+H
c=Rt+radar_alt
alpha=np.arccos((b**2+c**2-rho**2)/(2*b*c))
print("alpha (deg): ", alpha*180/np.pi)
coords_radar = CoordinatesWGS84(radar_lat, radar_lon, radar_alt)
print("Radar coords (lat, lon, alt):",radar_lat, radar_lon, radar_alt)
print(str(coords_radar))
coords_p=CoordinatesWGS84(radar_lat+alpha*np.sin(theta),radar_lon+alpha*np.cos(theta),H)
print(str(coords_p))
print("New point coords (lat, lon, alt):",radar_lat+alpha*180*np.sin(theta)/np.pi,radar_lon+alpha*180*np.cos(theta)/np.pi,H)

alpha (deg):  0.028267488844797697

Radar coords (lat, lon, alt): 0.7208306773472802 0.03668585124955859 27

41:18:2.0000N 002:06:7.0000E 27.0000m
lat:41.300555556 lon:2.101944444

41:17:2.7280N 002:04:44.2804E 68.5800m
lat:41.284091108 lon:2.078966784

New point coords (lat, lon, alt): 0.7043662298588637 0.013708190478889694 68.58